# **CASE: SATISFAÇÃO EM RESTAURANTE**

---

### Instalação de bibliotecas

In [ ]:
# Para geração de correlograma
install.packages("GGally")
# Para cálculo do VIF
install.packages("car")

### Carregamento de bibliotecas

In [ ]:
library(GGally)
library(car)

### Leitura da base de dados

In [ ]:
dados_restaurante <- read.table(file   = "Satisfacao_Restaurante.txt",
                                sep    = "\t",
                                dec    = ".",
                                header = TRUE)

### Visualização da base de dados

In [ ]:
View(dados_restaurante)

### Análise exploratória breve

In [ ]:
# Medidas resumo
summary(dados_restaurante)

In [ ]:
# Tabela de frequências
table(dados_restaurante$FLAG_LITORAL)
prop.table(table(dados_restaurante$FLAG_LITORAL))

### Análise bivariada: variável resposta vs. variáveis explicativas

In [ ]:
# Comando auxiliar para aumentar resolução do gráfico (recomendado apenas no Colab)
reset_options <- options()
options(repr.plot.width = 15, repr.plot.height = 15)

In [ ]:
# Correlograma
ggpairs(dados_restaurante[, !names(dados_restaurante) %in% c("ID_UNIDADE", "FLAG_LITORAL")],
        title = "Correlograma",
        upper = list(continuous = wrap("cor", size = 5)))

In [ ]:
# Boxplot para variável qualitativa FLAG_LITORAL
boxplot(dados_restaurante$QTDE_CLIENTES_DEZ ~ dados_restaurante$FLAG_LITORAL,
        main = "Boxplot de QTDE_CLIENTES_DEZ por FLAG_LITORAL")

In [ ]:
# Resetando parâmetros gráficos (recomendado apenas no Colab)
options(reset_options)

### Modelo de regressão linear múltipla

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [ ]:
# Ajuste do modelo 1: inicial
regressao_1 <- lm(QTDE_CLIENTES_DEZ ~
                  FLAG_LITORAL +
                  TEMPO_MED_ESPERA +
                  VALOR_MED_CONTA +
                  NOTA_MED_COMIDA +
                  NOTA_MED_ATENDIMENTO +
                  NOTA_MED_AMBIENTE +
                  QTDE_MED_CLIENTES,
                data = dados_restaurante)

summary(regressao_1)

In [ ]:
# Ajuste do modelo 2: retirando VALOR_MED_CONTA
regressao_2 <- lm(QTDE_CLIENTES_DEZ ~
                  FLAG_LITORAL +
                  TEMPO_MED_ESPERA +
                  NOTA_MED_COMIDA +
                  NOTA_MED_ATENDIMENTO +
                  NOTA_MED_AMBIENTE +
                  QTDE_MED_CLIENTES,
                data = dados_restaurante)

summary(regressao_2)

In [ ]:
# Avaliação de colinearidade no modelo 2
data.frame(VIF = vif(regressao_2))

### Intervalos de confiança

In [ ]:
# Escolha do beta (0 para intercepto, ou 1, 2, ... para os ângulos referentes a cada variável explicativa)
beta = 0

In [ ]:
# Cálculo do intervalo
print("Limite inferior")
as.numeric(regressao_2$coefficients[beta + 1] - 1.96 * coef(summary(regressao_2))[beta + 1, "Std. Error"])
print("Limite superior")
as.numeric(regressao_2$coefficients[beta + 1] + 1.96 * coef(summary(regressao_2))[beta + 1, "Std. Error"])

### Qualidade de ajuste

In [ ]:
# Coeficiente de determinação ajustado (R² ajustado)
summary(regressao_2)$adj.r.squared

In [ ]:
# MAE
sum(abs(regressao_2$residuals))/nrow(dados_restaurante)

In [ ]:
# MAPE
sum(abs(regressao_2$residuals/dados_restaurante$QTDE_CLIENTES_DEZ))/nrow(dados_restaurante)

### Análise de resíduos

In [ ]:
# Histograma dos resíduos
hist(x      = regressao_2$residuals,
     main   = "Histograma dos resíduos",
     col    = "darkturquoise",
     border = "white")

In [ ]:
# QQ plot dos resíduos
qqnorm(y    = regressao_2$residuals,
       main = "QQ-plot dos resíduos",
       col  = "darkturquoise",
       pch  = 19)
qqline(y    = regressao_2$residuals,
       col  = "red")

In [ ]:
# Resíduos vs. valores preditos
plot(x    = regressao_2$fitted.values,
     y    = regressao_2$residuals,
     pch  = 19,
     col  = "darkturquoise",
     xlab = "Valores ajustados",
     ylab = "Resíduos",
     main = "Resíduos vs. valores ajustados")

### Exemplo de uso (aplicação) do modelo

In [ ]:
# Criação de base de dados com uma nova unidade no litoral, que recebeu em média 3.000 clientes por mês de jan a nov,
# tem nota média igual a 9 para os três aspectos avaliados e tempo de espera médio de 20’
# Obs.: os nomes das colunas devem ser idênticos aos da base utilizada para construção do modelo
novos_dados <- data.frame(FLAG_LITORAL         = c(1),
                          TEMPO_MED_ESPERA     = c(20),
                          QTDE_MED_CLIENTES    = c(3000),
                          NOTA_MED_COMIDA      = c(9),
                          NOTA_MED_ATENDIMENTO = c(9),
                          NOTA_MED_AMBIENTE    = c(9))

In [ ]:
# Aplicação do modelo
novos_dados$QTDE_CLIENTES_DEZ_PREDITO <- predict(regressao_2, novos_dados)
View(novos_dados)